In [41]:
# Import libraries
import pandas as pd
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext, SparkSession
from pyspark.sql.types import StructType, StructField, DoubleType, IntegerType, StringType, ArrayType, FloatType
import re
import string
from pyspark.sql.functions import udf, col, size, lit, explode, isnan, when, count, min, max, struct
from pyspark.ml.feature import HashingTF, IDF, Tokenizer, IndexToString, StringIndexer, VectorIndexer, CountVectorizer
from collections import Counter
import networkx as nx
import nltk
from nltk.corpus import words
import gensim
import gensim.corpora as corpora
from gensim.models import CoherenceModel
from gensim.models.ldamulticore import LdaMulticore
import matplotlib.pyplot as plt
%matplotlib inline
import openai
import json
import ast
from igraph import *
import datashader as ds
import datashader.transfer_functions as tf
from datashader.layout import random_layout, circular_layout, forceatlas2_layout
from datashader.bundling import connect_edges, hammer_bundle
from textblob import TextBlob
import numpy as np
import os
os.chdir('/home/dataguy/Documents/')
from tweetology import establish_conn
pd.set_option("display.max_columns", None)
import neo4jupyter
from py2neo import Graph, Node, Relationship
neo4jupyter.init_notebook_mode()
from igraph import *
import graphistry

<IPython.core.display.Javascript object>

In [3]:
# Initialize spark
sc = SparkContext.getOrCreate(SparkConf().setMaster("local[*]"))
spark = SparkSession.builder.getOrCreate()

In [30]:
# Read
df = spark.read.json('/home/dataguy/data/us_news_tweets.json')
df1 = df.withColumn('username', col('user.username')).withColumn('country', col('place.country')).withColumn('country_cd', col('place.countryCode'))
df2 = df1.withColumn('quoted', col('quotedTweet.user.username')).withColumn('mentions', col('mentionedUsers.username')).withColumn('reply_to', col('inReplyToUser.username')).drop('inReplyToUser', 'mentionedUsers', 'renderedContent', 'id', 'media', 'outlinks', '_type', 'cashtags', 'conversationId', 'inReplyToTweetId', 'source', 'sourceUrl', 'sourceLabel', 'tcooutlinks', 'url', 'country', 'lang', 'retweetedTweet', 'user', 'coordinates', 'place', 'quotedTweet').filter("country == 'United States'")
df3 = df2.toPandas()
df4 = df3.set_index(pd.DatetimeIndex(df3['date'])).drop('date', axis = 1)

In [31]:
df4.head()

,content,hashtags,likeCount,quoteCount,replyCount,retweetCount,username,country_cd,quoted,mentions,reply_to
date,,,,,,,,,,,
2022-08-16 09:49:10+00:00,#News #Singularity #Ponzi/#RP \nTo: #Teamer\nC...,"[News, Singularity, Ponzi, RP, Teamer, Bloodli...",1,0,1,0,TheGodSeek,US,None,None,TheGodSeek
2022-08-16 07:32:15+00:00,#News #Singularity #Ponzi/#RP\nAnyone can fiel...,"[News, Singularity, Ponzi, RP, HeadHoncho]",1,0,0,1,TheGodSeek,US,None,None,TheGodSeek
2022-08-16 07:30:55+00:00,#News #Singularity #Ponzi/#RP\nWhen you're wal...,"[News, Singularity, Ponzi, RP]",1,0,1,1,TheGodSeek,US,None,None,TheGodSeek
2022-08-16 07:28:28+00:00,#News #Singularity #Ponzi/#RP\nUmm... you take...,"[News, Singularity, Ponzi, RP]",1,0,1,1,TheGodSeek,US,TheGodSeek,None,None
2022-08-16 07:22:33+00:00,#News #Singularity #Ponzi/#RP\nThis is a dynam...,"[News, Singularity, Ponzi, RP, UpTop, HeadHonc...",1,1,0,1,TheGodSeek,US,None,None,TheGodSeek


In [32]:
users = df2.select('username').distinct().collect()
quoted = df2.select('quoted').distinct().collect()
mentions = df2.select('mentions').distinct().collect()

# Separate edges
mnts = []

# Iterate mentions
for x in mentions:
    
    if x[0]:
        
        for y in x[0]:
        
            mnts.append(y)
            
nodes = [x[0] for x in users] + [y[0] for y in quoted] + [i for i in (mnts)]
nodes = [i for i in set(nodes)]

In [42]:
edges = []

# Iterate data to build network connections
for x in df2.select('username', 'quoted', 'mentions', 'reply_to').collect():
    
    if x[1]:
        
        edges.append((x[0], x[1], 'quote'))
        
    if x[2]:
        
        for y in x[2]:
            
            edges.append((x[0], y, 'mention'))
            
    if x[3]:
        
        edges.append((x[0], x[3], 'reply'))

In [46]:
edges[0:10]

[('TheGodSeek', 'TheGodSeek', 'reply'),
 ('TheGodSeek', 'TheGodSeek', 'reply'),
 ('TheGodSeek', 'TheGodSeek', 'reply'),
 ('TheGodSeek', 'TheGodSeek', 'quote'),
 ('TheGodSeek', 'TheGodSeek', 'reply'),
 ('TheGodSeek', 'TheGodSeek', 'quote'),
 ('TheGodSeek', 'TheGodSeek', 'quote'),
 ('TheGodSeek', 'TheGodSeek', 'reply'),
 ('TheGodSeek', 'TheGodSeek', 'quote'),
 ('JeffLennox', 'MiamiDadePD', 'mention')]

In [44]:
e = pd.DataFrame(edges, columns = ['source', 'target', 'type'])
e1 = e.groupby(['source', 'target', 'type']).size().reset_index().rename({0: 'weight'}, axis = 1)
e2 = e1[e1['source'].str.strip() != e1['target'].str.strip()]

In [45]:
e2.head()

,source,target,type,weight
0,0arcadian0,CBSMornings,quote,1
1,1025TheBone,TheJohnSenning,mention,1
2,1Brandycampbell,foxweather,quote,1
3,239SpiritWakeUp,afronation,mention,1
4,43HarvardSt,Lofan_loogle,mention,1


In [38]:
g = graphistry.bind(source = "source", destination = "target")
g.edges(e2).plot()

In [51]:
# Plot with igraph
e = [(x[0], x[1]) for x in edges]
g = Graph(directed = True)
g.add_vertices(nodes)
g.add_edges(e)
plot(g)

In [ ]:
for e in g.es:
    src, targ = e.tuple
    if g.are_connected(targ, src):
        e["color"] = "green"           #follow + followback
        e["arrow_size"] = 0                       
    else:
        e["color"] = "blue"            #follow
        e["arrow_size"] = 1
plot(g, **style)

In [53]:
nodes[0:10]

['FEWCoalition',
 'NicholasBlack60',
 'ShahidaMunir11',
 'JeffreyLustick',
 'NYMag',
 'StephenAtHome',
 'McIlroyRory',
 'Honchodm',
 'bubbaprog',
 'JoeBiden']

In [52]:
from graph_tool.all import *

In [57]:
# Init
g = Graph()

In [63]:
# Add vertices and edges
for x in edges[0:50]:
    
    y = g.add_vertex(x[0])
    z = g.add_vertex(x[1])
    
    out = g.add_edge(y, z)

ArgumentError: Python argument types in
    graph_tool.libgraph_tool_core.add_vertex(GraphInterface, str)
did not match C++ signature:
    add_vertex(graph_tool::GraphInterface {lvalue}, unsigned long)

In [62]:
x[0]

'TheGodSeek'

In [61]:
x[1]

'TheGodSeek'